In [2]:
import torch
import torch.nn as nn
import torchaudio
from torchvision import transforms as vis_transforms
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from config import *
import librosa.display as libdis
import IPython.display as ipd


/Users/somaf/Documents/python-environments/tp_project_env/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(



## Model user





In [4]:
date = '2021-04-05'
run_id = '3'
epoch = '30'
model_path = f"data_interpreter/{date}/run_{run_id}/efinal_model.pt"
config = SegmentedCNNconfig()

In [5]:
model = config.model()
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))


RuntimeError: Error(s) in loading state_dict for SegmentedCNN:
	size mismatch for fc1.weight: copying a param with shape torch.Size([10, 16384]) from checkpoint, the shape in current model is torch.Size([10, 57600]).

## Data prep

In [17]:
genre = 'blues'
number = '00001'

music_path = f"data/WAV/{genre}/{genre}.{number}.wav"

wd, sr = torchaudio.load(music_path, normalization=True)
resampler = aud_transforms.Resample(sr, 16000)
audio = resampler(wd).squeeze()
# wd.shape
sr, audio.shape

(22050, torch.Size([480214]))

In [10]:
dataset_params = dict(
        frames=256,
        bands=128,
        window_size=1024,
        hop_size=256,
        e0=1e-3
    )
mel_spec_transform = nn.Sequential(
                aud_transforms.MelSpectrogram(
                        sample_rate=16000,
                        n_mels=dataset_params["bands"],
                        n_fft=dataset_params["window_size"],
                        hop_length=dataset_params["hop_size"]
                ),
                aud_transforms.AmplitudeToDB()
        )

In [32]:
def splitsongs(wd, overlap = 0.25):
    temp_X = []

    # Get the input song array size
    xshape = wd.shape[0]
    chunk = 48000 # min wave arr len is 478.912 --> 10 chunks without overlap
    offset = int(chunk*(1.-overlap))
    
    # Split the song and create new ones on windows
    spsong = [wd[i:i+chunk] for i in range(0, xshape - chunk + offset, offset)]
    for s in spsong:
        if s.shape[0] != chunk:
            continue

        temp_X.append(s)

    return np.array(temp_X)


In [34]:
patches = splitsongs(audio[:478912])
print(type(patches))
print(len(patches))
mel_specs = [mel_spec_transform(patch) for patch in patches]

final_patches = torch.stack(mel_specs)

# lmao = mel_spec_transform(patches[0])
# mel_spec = mel_spec_transform(audio[:465984]) # 456 width
# patches = torch.split(mel_spec, 76, dim=1)
# final_patches = torch.stack(patches)
# lmao.shape

<class 'numpy.ndarray'>
12
<ipython-input-32-c6bd5038b9e8>:17: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  return np.array(temp_X)
<ipython-input-32-c6bd5038b9e8>:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(temp_X)


In [35]:
final_patches.shape
# libdis.specshow(final_patches[4,:,:].numpy())


torch.Size([12, 128, 188])

In [52]:
def speedx(signal, factor):
    """ Multiplies the sound's speed by some `factor` """
    indices = torch.round( torch.range(0, len(signal), factor) )
    indices = indices[indices < len(signal)]
    return sound_array[ indices ]

In [51]:
def stretch(sound_array, f, window_size, h):
    """ Stretches the sound by a factor `f` """

    phase  = np.zeros(window_size)
    hanning_window = np.hanning(window_size)
    result = np.zeros( len(sound_array) /f + window_size)

    for i in np.arange(0, len(sound_array)-(window_size+h), h*f):

        # two potentially overlapping subarrays
        a1 = sound_array[i: i + window_size]
        a2 = sound_array[i + h: i + window_size + h]

        # resynchronize the second array on the first
        s1 =  np.fft.fft(hanning_window * a1)
        s2 =  np.fft.fft(hanning_window * a2)
        phase = (phase + np.angle(s2/s1)) % 2*np.pi
        a2_rephased = np.fft.ifft(np.abs(s2)*np.exp(1j*phase))

        # add to result
        i2 = int(i/f)
        result[i2 : i2 + window_size] += hanning_window*a2_rephased

    result = ((2**(16-4)) * result/result.max()) # normalize (16bit)

    return result.astype('int16')


In [53]:
def pitchshift(snd_array, n, window_size=2**13, h=2**11):
    """ Changes the pitch of a sound by ``n`` semitones. """
    factor = 2**(1.0 * n / 12.0)
    stretched = stretch(snd_array, 1.0/factor, window_size, h)
    return speedx(stretched[window_size:], factor)


In [54]:
# lib_gen_ps = librosa.effects.pitch_shift(row.detach().numpy(), BASE_SAMPLE_RATE, (1 + factor[index]))

In [76]:
loss = nn.CrossEntropyLoss()

In [77]:
temp_pred = torch.tensor([0, 0, 1])
temp_target = 2
loss(temp_pred, temp_target) 

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)